## Data Cleaning

Now since we have extracted data from the website, it is not cleaned and ready to be analyzed yet. The reviews section will need to be cleaned for punctuations, spellings and other characters. 

In [15]:
#imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#regex
import re

import nltk
nltk.download('stopwords')

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSI_PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MSI_PC\AppData\Roaming\nltk_data...


True

In [16]:
#create a dataframe from csv file

cwd = os.getcwd()

df = pd.read_csv(cwd+"/BA_reviews.csv", index_col=0)

In [17]:
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | This flight was one of the ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,21st January 2023,United Kingdom
1,Not Verified | It seems that there is a race t...,2,19th January 2023,United States
2,Not Verified | As a Spanish born individual l...,3,19th January 2023,United Kingdom
3,✅ Trip Verified | A rather empty and quiet fl...,2,18th January 2023,United Kingdom
4,✅ Trip Verified | Easy check in and staff mem...,9,17th January 2023,United Kingdom


We will also create a column which mentions if the user is verified or not. 

In [18]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [19]:
df['verified']

0        True
1       False
2       False
3        True
4        True
        ...  
3413    False
3414    False
3415    False
3416    False
3417    False
Name: verified, Length: 3418, dtype: bool

### Cleaning Reviews

We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [20]:
#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()


reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [21]:
# add the corpus to the original dataframe

df['corpus'] = corpus

In [22]:
df.head()

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | This flight was one of the ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,21st January 2023,United Kingdom,True,flight one worst ever life wanted pamper bough...
1,Not Verified | It seems that there is a race t...,2,19th January 2023,United States,False,verified seems race bottom amongst airline can...
2,Not Verified | As a Spanish born individual l...,3,19th January 2023,United Kingdom,False,verified spanish born individual living englan...
3,✅ Trip Verified | A rather empty and quiet fl...,2,18th January 2023,United Kingdom,True,rather empty quiet flight tel aviv friendly ca...
4,✅ Trip Verified | Easy check in and staff mem...,9,17th January 2023,United Kingdom,True,easy check staff member polite helpful made sp...


### Cleaning/Fromat date

In [23]:
df.dtypes

reviews     object
stars       object
date        object
country     object
verified      bool
corpus      object
dtype: object

In [24]:
# convert the date to datetime format

df.date = pd.to_datetime(df.date)

In [25]:
df.date.head()

0   2023-01-21
1   2023-01-19
2   2023-01-19
3   2023-01-18
4   2023-01-17
Name: date, dtype: datetime64[ns]

### Cleaning ratings with stars

In [26]:
#check for unique values
df.stars.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '2', '3', '9', '1', '4', '5', '8',
       '6', '7', '10', 'None'], dtype=object)

In [27]:
# remove the \t and \n from the ratings
df.stars = df.stars.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [28]:
df.stars.value_counts()

1       743
2       384
3       382
8       345
10      304
7       299
9       291
5       255
4       232
6       178
None      5
Name: stars, dtype: int64

There are 5 rows having values "None" in the ratings. We will drop all these 5 rows. 

In [29]:
# drop the rows where the value of ratings is None
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)

In [30]:
#check the unique values again
df.stars.unique()

array(['5', '2', '3', '9', '1', '4', '8', '6', '7', '10'], dtype=object)

## Check for null Values

In [31]:
df.isnull().value_counts()

reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     3411
                       True     False     False        2
dtype: int64

In [32]:
df.country.isnull().value_counts()

False    3411
True        2
Name: country, dtype: int64

We have two missing values for country. For this we can just remove those two reviews (rows) from the dataframe. 

In [33]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [34]:
df.shape

(3411, 6)

In [35]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | This flight was one of the ...,5,2023-01-21,United Kingdom,True,flight one worst ever life wanted pamper bough...
1,Not Verified | It seems that there is a race t...,2,2023-01-19,United States,False,verified seems race bottom amongst airline can...
2,Not Verified | As a Spanish born individual l...,3,2023-01-19,United Kingdom,False,verified spanish born individual living englan...
3,✅ Trip Verified | A rather empty and quiet fl...,2,2023-01-18,United Kingdom,True,rather empty quiet flight tel aviv friendly ca...
4,✅ Trip Verified | Easy check in and staff mem...,9,2023-01-17,United Kingdom,True,easy check staff member polite helpful made sp...
...,...,...,...,...,...,...
3406,LHR-JTR-LHR. A tale of opposites for BA's new ...,10,2014-05-18,United Kingdom,False,lhr jtr lhr tale opposite ba new route santori...
3407,Gatwick-Venice return. Two contrasting experie...,2,2014-05-18,United Kingdom,False,gatwick venice return two contrasting experien...
3408,1st Class LHR-PHX. Totally seamless flight. Gr...,10,2014-05-18,United Kingdom,False,st class lhr phx totally seamless flight great...
3409,When we booked our holiday to Barbados I was r...,7,2014-05-18,United Kingdom,False,booked holiday barbados really pleased see fly...


*****

Now our data is all cleaned and ready for data visualization and data analysis.

In [36]:
# export the cleaned data

df.to_csv(cwd + "/cleaned-BA-reviews.csv")